In [48]:
#Importing packages

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [49]:
#base url from which we are going to traverse through multiple urls and store the result
main_url = "https://powrbot.com"
base_url = "https://powrbot.com/companies/list-of-industries/"
company = pd.DataFrame()

In [50]:
#sending GET request to the above url and checking for connectivity status

response = requests.get(base_url)
response.status_code

200

In [51]:
#html file content of the base_url is stored

html = response.content

In [52]:
#parsing through the full html page from the above url and storing the parsed document for future use

soup = BeautifulSoup(html, 'lxml')
with open('html_file.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))

In [53]:
def get_the_number_of_pages(soup, base_url):
    try:
        all_pages = [i['href'] for i in soup.find_all('ul', {"class":"pagination"})[0].find_all('a')[2:]]
    except:
        all_pages = 0
        return 0
    first_page = all_pages[0]
    last_page = all_pages[-1]
    first = int(first_page.split('=')[-1])
    last = int(last_page.split('=')[-1])+1
    pre_write = first_page.split('=')[0]+'='
    

    pages = []
    for number in range(first, last):
        pages.append(pre_write+str(number))

    
    all_pages_full_link = [base_url+url for url in pages]
    
    return(all_pages_full_link)

In [54]:
def get_all_links_from_industry_page(soup, link):
    #this is giving for landing page
    list_of_industries = soup.find_all("ul", {"class": "list-unstyled"})
    all_links_for_industries = [i['href'] for listed in list_of_industries for i in listed.find_all('a')]
    complete_links_for_industries = [main_url+url for url in all_links_for_industries]
    print('PAGE 1 DONE......')

    all_pages = get_the_number_of_pages(soup,link)

    if all_pages == 0:
        return complete_links_for_industries
        
    #below is for giving all the other pages
    length = len(all_pages)

    c = 0
    while c < (length):
        new_link = all_pages[c]

        response = requests.get(new_link)
        html = response.content
        soup = BeautifulSoup(html, 'lxml')
        list_of_industries = soup.find_all("ul", {"class": "list-unstyled"})
        all_links_for_industries = [i['href'] for listed in list_of_industries for i in listed.find_all('a')]
        for url in all_links_for_industries:
            complete_links_for_industries.append(main_url+url)
        c = c+1
        print('PAGE {} DONE......'.format(c+1))
 

    return complete_links_for_industries

In [56]:
all_links = get_all_links_from_industry_page(soup, base_url)

PAGE 1 DONE......
PAGE 2 DONE......
PAGE 3 DONE......
PAGE 4 DONE......
PAGE 5 DONE......
PAGE 6 DONE......
PAGE 7 DONE......
PAGE 8 DONE......
PAGE 9 DONE......


In [57]:
all_links

['https://powrbot.com/companies/list-of-companies-in-advertising-industry/',
 'https://powrbot.com/companies/list-of-companies-in-agricultural-products-industry/',
 'https://powrbot.com/companies/list-of-companies-in-airlines-industry/',
 'https://powrbot.com/companies/list-of-companies-in-airport-services-industry/',
 'https://powrbot.com/companies/list-of-companies-in-aluminum-industry/',
 'https://powrbot.com/companies/list-of-companies-in-application-software-industry/',
 'https://powrbot.com/companies/list-of-companies-in-automobiles-industry/',
 'https://powrbot.com/companies/list-of-companies-in-automotive-retail-industry/',
 'https://powrbot.com/companies/list-of-companies-in-banks-industry/',
 'https://powrbot.com/companies/list-of-companies-in-beverages-industry/',
 'https://powrbot.com/companies/list-of-companies-in-biotechnology-industry/',
 'https://powrbot.com/companies/list-of-companies-in-broadcasting-industry/',
 'https://powrbot.com/companies/list-of-companies-in-buil

I have created for - SOFTWARE, IT SERVICES, ADVERTISEMENT, BANK, MEDIA, FINANCE

In [63]:
#selecting one link from all the links and getting a dataframe ready for it

link_temp =  'https://powrbot.com/companies/list-of-companies-in-media-industry/'

#getting all the pages out of the selected link. These are link with the required company information
print('STARTED FOR LINK - ',link_temp)
response1 = requests.get(link_temp)
html1 = response1.content
soup1 = BeautifulSoup(html1, 'lxml')
temp_list = get_all_links_from_industry_page(soup1, link_temp)

STARTED FOR LINK -  https://powrbot.com/companies/list-of-companies-in-media-industry/
PAGE 1 DONE......
PAGE 2 DONE......
PAGE 3 DONE......
PAGE 4 DONE......
PAGE 5 DONE......
PAGE 6 DONE......
PAGE 7 DONE......
PAGE 8 DONE......
PAGE 9 DONE......
PAGE 10 DONE......
PAGE 11 DONE......
PAGE 12 DONE......
PAGE 13 DONE......
PAGE 14 DONE......
PAGE 15 DONE......


In [64]:
#creating a single dataframe out of single company details

def get_dataset_out_of_your_companies(soup, c):
    column = [i.find('th').string.strip('\n').strip() for i in soup.find_all('div', {'class':'g-my-20'})[0].find_all('tr')]
    values = [i.find('td').string for i in soup.find_all('div', {'class':'g-my-20'})[0].find_all('tr')]
    value_list = []
    for val in values:
        if val is not None:
            value_list.append(val.strip())
        else:
            value_list.append(0)
    df_val = dict(zip(column, value_list))
    small_df = pd.DataFrame(df_val, index = [c])
    small_df['INDUSTRY'] = 'DIGITAL MARKETING'
    return small_df

In [65]:
#creating a complete dataframe for all the companies listed under the selected industry

c = 1
df = pd.DataFrame()
for links in temp_list:
    print('STARTED FOR ', links)
    response1 = requests.get(links)
    html1 = response1.content
    soup1 = BeautifulSoup(html1, 'lxml')
    small_df = get_dataset_out_of_your_companies(soup1, c)
    c = c+1
    df = pd.concat([df,small_df], axis = 0)
    

STARTED FOR  https://powrbot.com/companies/profile/reliance-big-entertainment/
STARTED FOR  https://powrbot.com/companies/profile/lee-enterprises-incorporated/
STARTED FOR  https://powrbot.com/companies/profile/meredith-corporation/
STARTED FOR  https://powrbot.com/companies/profile/nielsen-nv/
STARTED FOR  https://powrbot.com/companies/profile/global-news/
STARTED FOR  https://powrbot.com/companies/profile/digiday/
STARTED FOR  https://powrbot.com/companies/profile/mother-nature-network/
STARTED FOR  https://powrbot.com/companies/profile/north-media/
STARTED FOR  https://powrbot.com/companies/profile/berlingske-media/
STARTED FOR  https://powrbot.com/companies/profile/sjllandske-medier/
STARTED FOR  https://powrbot.com/companies/profile/ubm/
STARTED FOR  https://powrbot.com/companies/profile/mediaset-espana/
STARTED FOR  https://powrbot.com/companies/profile/nielsen-holdings/
STARTED FOR  https://powrbot.com/companies/profile/naspers/
STARTED FOR  https://powrbot.com/companies/profile

In [66]:
df

,Company Name,Website,Snippet,Type,Parent,Founded,Formerly,Industry,Products,Key people,...,Founders,Predecessor,Trading name,Divisions,Brands,Alexa rank,Native name,Owners,Predecessors,display_url
1,Reliance Big Entertainment,Reliance Entertainment,"Reliance Entertainment, in association with Bi...",Division,Reliance Anil Dhirubhai Ambani Group,2005,"Reliance BIG Entertainment (2005–2009), BIG Pi...","Entertainment, Mass media","Films, Web series, Animation, Digital, Gaming",Anil Ambani(Chairman)Tina Ambani (Director),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Lee Enterprises, Incorporated",Lee Enterprises website,"As of March 16, 2020 BH Media Group is part of...",Public,NaN,1890,NaN,Media,Newspapers and television station,Mary Junck(Executive Chairman)Kevin Mowbray(Pr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Meredith Corporation,www.meredith.com,Your browser does not support the video tag. ©...,Public,NaN,1902,NaN,Mass media,"Newspapers, Magazines, Television, Educational...","Steve Lacy, (Executive Chairman), Tom Harty, (...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nielsen N.V.,www.nielsen.com,global.nielsen.com,Public limited company,NaN,1923,NaN,Media,Consumer informationConsumer researchMarket me...,James Attwood(non-executive Chairman)[1]David ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Global News,GlobalNews.ca,Global News is the news and current affairs di...,Department of the Global Television Network,Global Television Network,1994,NaN,"Media, News",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Tamedia,www.tx.group,Leo Eiholzer gewinnt das Förderprogramm für in...,Public,NaN,1893,NaN,Media,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
140,Reliance Entertainment,Reliance Entertainment,Reliance Entertainment Pvt Ltd is a division o...,Division,Reliance Anil Dhirubhai Ambani Group,2005,Reliance BIG Entertainment,Media,FilmWeb Series & TVAnimationDigitalGaming,Anil Ambani (chairman)Tina Ambani (Director),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,Jim Pattison Group,www.jimpattison.com,The Jim Pattison Group is a Vancouver-based co...,Private,NaN,1961,NaN,"Media, automotive, packaging, food sales and d...","Media, Broadcasting, Grocery, Billboards","Jim Pattison, chairman and CEOGlen Clark, Pres...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,Unidad Editorial,unidadeditorial.es,,NaN,NaN,,NaN,media production,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unidadeditorial.es


In [68]:
company = pd.concat([company, df], axis = 0)

In [73]:
cols_save = ['Company Name', 'Website', 'Type', 'Parent', 'Founded', 'INDUSTRY']

In [74]:
company[cols_save].to_csv('advertise.csv')